In [1]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import col, sum, concat_ws, collect_list

In [2]:
# CREATE SPARK SESSION (DEPRECATED BUT WORKING)

appName = "PySpark SQL Server Example - via JDBC"
master = "local"
conf = SparkConf().setAppName(appName).setMaster(master).set("spark.driver.extraClassPath", "C:/spark/sqljdbc_12.8/enu/jars/mssql-jdbc-12.8.1.jre11.jar")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

c:\Users\sudip\anaconda3\envs\suddata\Lib\site-packages\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
# VARIABLES NEEDED TO MAKE CONNECTION AND TABLE NAMES
database = "Chinook"
user = "SUDIPTOMW7"
password = "password"
jdbc_url = f"jdbc:sqlserver://localhost:1433;databaseName={database};encrypt=true;trustServerCertificate=true"

album, artist, customer, employee, genre, invoice, invoice_line, media_type, playlist, playlist_track, track = "dbo.Album", "dbo.Artist",\
    "dbo.Customer", "dbo.Employee", "dbo.Genre",\
    "dbo.Invoice", "dbo.InvoiceLine", "dbo.MediaType",\
    "dbo.Playlist", "dbo.PlaylistTrack", "dbo.Track"

In [4]:
# CREATE DF FOR EACH TABLE USING JDBC

album_df = spark.read.format("jdbc").option("url", jdbc_url).option("dbtable", album).option("user", user)\
    .option("password", password).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").load()

artist_df = spark.read.format("jdbc").option("url", jdbc_url).option("dbtable", artist).option("user", user)\
    .option("password", password).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").load()

customer_df = spark.read.format("jdbc").option("url", jdbc_url).option("dbtable", customer).option("user", user)\
    .option("password", password).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").load()

employee_df = spark.read.format("jdbc").option("url", jdbc_url).option("dbtable", employee).option("user", user)\
    .option("password", password).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").load()

genre_df = spark.read.format("jdbc").option("url", jdbc_url).option("dbtable", genre).option("user", user)\
    .option("password", password).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").load()

invoice_df = spark.read.format("jdbc").option("url", jdbc_url).option("dbtable", invoice).option("user", user)\
    .option("password", password).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").load()

invoice_line_df = spark.read.format("jdbc").option("url", jdbc_url).option("dbtable", invoice_line).option("user", user)\
    .option("password", password).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").load()

media_type_df = spark.read.format("jdbc").option("url", jdbc_url).option("dbtable", media_type).option("user", user)\
    .option("password", password).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").load()

playlist_df = spark.read.format("jdbc").option("url", jdbc_url).option("dbtable", playlist).option("user", user)\
    .option("password", password).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").load()

playlist_track_df = spark.read.format("jdbc").option("url", jdbc_url).option("dbtable", playlist_track).option("user", user)\
    .option("password", password).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").load()

track_df = spark.read.format("jdbc").option("url", jdbc_url).option("dbtable", track).option("user", user)\
    .option("password", password).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").load()


EASY

In [53]:
# 1. Retrieve the total sales (invoiced amount) for each customer.

q1_df = customer_df.alias("c").join(invoice_df.alias("i"), col("c.CustomerId") == col("i.CustomerId"), "inner")\
    .groupBy(col("c.CustomerId").alias("Customer"))\
    .agg(sum(col("i.Total")).alias("Total_Sales"))\
    .show()

+--------+-----------+
|Customer|Total_Sales|
+--------+-----------+
|      31|      37.62|
|      53|      37.62|
|      34|      39.62|
|      28|      43.62|
|      27|      37.62|
|      26|      47.62|
|      44|      41.62|
|      12|      37.62|
|      22|      39.62|
|      47|      37.62|
|       1|      39.62|
|      52|      37.62|
|      13|      37.62|
|      16|      37.62|
|       6|      49.62|
|       3|      39.62|
|      40|      38.62|
|      20|      39.62|
|      57|      46.62|
|      54|      37.62|
+--------+-----------+
only showing top 20 rows



In [61]:
# 2. List all albums by a specific artist (e.g., “Queen”).

q2_df = artist_df.alias("ar").join(album_df.alias("al"), col("ar.ArtistId") == col("al.ArtistId"), "inner")\
    .groupBy(col("ar.ArtistId"), col("ar.Name"))\
    .agg(concat_ws(", ", collect_list(col("al.Title"))).alias("Albums"))\
    .show()

+--------+--------------------+--------------------+
|ArtistId|                Name|              Albums|
+--------+--------------------+--------------------+
|       1|               AC/DC|For Those About T...|
|       2|              Accept|Balls to the Wall...|
|       3|           Aerosmith|            Big Ones|
|       4|   Alanis Morissette|  Jagged Little Pill|
|       5|     Alice In Chains|            Facelift|
|       6|Antônio Carlos Jobim|Warner 25 Anos, C...|
|       7|        Apocalyptica|Plays Metallica B...|
|       8|          Audioslave|Audioslave, Out O...|
|       9|            BackBeat| BackBeat Soundtrack|
|      10|        Billy Cobham|The Best Of Billy...|
|      11| Black Label Society|Alcohol Fueled Br...|
|      12|       Black Sabbath|Black Sabbath, Bl...|
|      13|          Body Count|          Body Count|
|      14|     Bruce Dickinson|    Chemical Wedding|
|      15|           Buddy Guy|The Best Of Buddy...|
|      16|      Caetano Veloso|Prenda Minha, S

In [65]:
# 3. Get a list of tracks from a specific genre (e.g., “Rock”).

q3_df = track_df.alias("t").join(genre_df.alias("g"), col("t.GenreId")==col("g.GenreId"), "inner")\
    .groupBy(col("g.name").alias("Genre")).agg(concat_ws("; ", collect_list(col("t.name"))).alias("Tracks"))\
    .show()

+-----------------+--------------------+
|            Genre|              Tracks|
+-----------------+--------------------+
|            World|Pura Elegancia; C...|
|      Hip Hop/Rap|ZeroVinteUm; Quei...|
|         TV Shows|Occupation / Prec...|
|       Bossa Nova|Samba Da Bênção; ...|
|            Latin|Jorge Da Capadóci...|
| Sci Fi & Fantasy|Crossroads, Pt. 1...|
|            Metal|Enter Sandman; Ma...|
|             Rock|For Those About T...|
|             Jazz|Desafinado; Garot...|
|         R&B/Soul|Please Please Ple...|
|            Drama|Don't Look Back; ...|
|       Soundtrack|Vai-Vai 2001; X-9...|
|Electronica/Dance|Just Another Stor...|
|        Classical|Symphony No. 3 in...|
|            Blues|First Time I Met ...|
|      Alternative|War Pigs; Say Hel...|
|    Rock And Roll|Money; Long Tall ...|
|  Science Fiction|Battlestar Galact...|
|              Pop|Dig-Dig, Lambe-La...|
|      Heavy Metal|Wildest Dreams; R...|
+-----------------+--------------------+
only showing top

In [ ]:
# 4. Find the total number of invoices for each customer.



In [ ]:
# 5 .Display the average track length (milliseconds) for each album.



In [ ]:
# 6. Retrieve all customers from the “USA” and their invoices.



In [ ]:
# 7. Show the total number of tracks in each genre.



In [ ]:
# 8 .List all tracks from albums released in a specific year (e.g., 2010).



In [ ]:
# 9. Find customers who have not placed any invoices.



In [ ]:
# 10. Retrieve the most popular genres by total number of tracks.



In [5]:
#spark.stop()